In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Load dataset
df = pd.read_csv("../data/processed/accepted_cleaned.csv")

# Clean and transform
df['interest_rate'] = pd.to_numeric(df['int_rate'].astype(str).str.replace('%', ''), errors='coerce') / 100
df['log_total_balance'] = np.log1p(df['total_balance'])

# Drop NAs and infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(subset=[
    'credit_utilization', 'credit_history_years', 'total_delinquency',
    'recent_inquiries', 'credit_mix', 'log_total_balance',
    'fico_score', 'interest_rate'
], inplace=True)

# Define features and targets
features = [
    'credit_utilization', 'credit_history_years', 'total_delinquency',
    'recent_inquiries', 'credit_mix', 'log_total_balance'
]
targets = ['fico_score', 'interest_rate']

X = df[features]
y = df[targets]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split for better generalization assessment
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train MultiOutput XGBoost
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
multi_model = MultiOutputRegressor(xgb)
multi_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = multi_model.predict(X_test)

for i, target in enumerate(targets):
    print(f"\nTarget: {target}")
    print(f"MAE: {mean_absolute_error(y_test.iloc[:, i], y_pred[:, i]):.4f}")
    print(f"R²: {r2_score(y_test.iloc[:, i], y_pred[:, i]):.4f}")



Target: fico_score
MAE: 17.6410
R²: 0.5256

Target: interest_rate
MAE: 0.0342
R²: 0.1665


In [2]:


# Load data
df = pd.read_csv("../data/processed/accepted_cleaned.csv")

# Handle interest rate and log balance
if 'interest_rate' not in df.columns and 'int_rate' in df.columns:
    df['interest_rate'] = pd.to_numeric(df['int_rate'].astype(str).str.replace('%', ''), errors='coerce') / 100
if 'log_total_balance' not in df.columns and 'total_balance' in df.columns:
    df['log_total_balance'] = np.log1p(df['total_balance'])

# Drop NA and infinite values
required_cols = [
    'credit_utilization', 'credit_history_years', 'total_delinquency',
    'recent_inquiries', 'credit_mix', 'log_total_balance',
    'fico_score', 'interest_rate'
]
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=required_cols)

# Feature and targets
X = df[['credit_utilization', 'credit_history_years', 'total_delinquency',
        'recent_inquiries', 'credit_mix', 'log_total_balance']]
y_fico = df['fico_score']
y_rate = df['interest_rate']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train separate XGBoost models
fico_model = XGBRegressor(
    subsample=1.0, reg_lambda=2, reg_alpha=0.5, n_estimators=200,
    max_depth=3, learning_rate=0.1, gamma=1, colsample_bytree=1.0,
    random_state=42, verbosity=0
)
rate_model = XGBRegressor(
    subsample=1.0, reg_lambda=1, reg_alpha=0.3, n_estimators=150,
    max_depth=4, learning_rate=0.05, gamma=0.5, colsample_bytree=1.0,
    random_state=42, verbosity=0
)

fico_model.fit(X_scaled, y_fico)
rate_model.fit(X_scaled, y_rate)

# Predict and evaluate
y_pred_fico = fico_model.predict(X_scaled)
y_pred_rate = rate_model.predict(X_scaled)

print("Target: FICO Score")
print("MAE:", round(mean_absolute_error(y_fico, y_pred_fico), 4))
print("R² :", round(r2_score(y_fico, y_pred_fico), 4))

print("\n Target: Interest Rate")
print("MAE:", round(mean_absolute_error(y_rate, y_pred_rate), 4))
print("R² :", round(r2_score(y_rate, y_pred_rate), 4))


Target: FICO Score
MAE: 16.5684
R² : 0.5179

 Target: Interest Rate
MAE: 0.0305
R² : 0.1313
